# 🎓 Práctica de Machine Learning: Predicción de Rendimiento Académico

**Estudiante:** [Tu Nombre]

**Fecha:** Diciembre 2024

## Objetivos

1. Implementar un modelo supervisado de clasificación para predecir aprobación/reprobación
2. Aplicar clustering (K-means) para identificar grupos de estudiantes
3. Comparar y evaluar ambos modelos

## 1. Importación de Librerías

In [ ]:
# Librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.cluster import KMeans

print("✅ Librerías importadas correctamente")

## 2. Carga y Exploración de Datos

In [ ]:
# Cargar dataset
df = pd.read_csv('data/academic_performance_master.csv')

print(f"📊 Dimensiones del dataset: {df.shape}")
print(f"📋 Columnas: {df.shape[1]}")
print(f"👥 Registros: {df.shape[0]}")

In [ ]:
# Vista previa
df.head(10)

In [ ]:
# Información del dataset
df.info()

In [ ]:
# Estadísticas descriptivas
df.describe()

### 2.1 Análisis de Calidad de Datos

In [ ]:
# Valores nulos
print("🔍 Valores Nulos por Columna:")
nulos = df.isnull().sum()
print(nulos[nulos > 0])

if nulos.sum() == 0:
    print("\n✅ No se encontraron valores nulos")
else:
    print(f"\n⚠️ Total de valores nulos: {nulos.sum()}")

In [ ]:
# Valores duplicados
duplicados = df.duplicated().sum()
print(f"Filas duplicadas: {duplicados}")

if duplicados == 0:
    print("✅ No se encontraron duplicados")
else:
    print(f"⚠️ Se encontraron {duplicados} filas duplicadas")

### 2.2 Visualización de la Distribución de Notas

In [ ]:
# Distribución de notas finales
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma
axes[0].hist(df['Nota_final'].dropna(), bins=25, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].axvline(df['Nota_final'].mean(), color='red', linestyle='--', linewidth=2, label=f'Media: {df["Nota_final"].mean():.2f}')
axes[0].axvline(df['Nota_final'].median(), color='green', linestyle='--', linewidth=2, label=f'Mediana: {df["Nota_final"].median():.2f}')
axes[0].set_title('Distribución de Notas Finales', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Nota Final')
axes[0].set_ylabel('Frecuencia')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Boxplot
axes[1].boxplot(df['Nota_final'].dropna())
axes[1].set_title('Boxplot de Notas Finales', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Nota Final')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Estadísticas
print(f"📊 Estadísticas de Nota Final:")
print(f"  - Mínimo: {df['Nota_final'].min():.2f}")
print(f"  - Máximo: {df['Nota_final'].max():.2f}")
print(f"  - Media: {df['Nota_final'].mean():.2f}")
print(f"  - Mediana: {df['Nota_final'].median():.2f}")
print(f"  - Desv. Estándar: {df['Nota_final'].std():.2f}")

## 3. Preparación de Datos

### 3.1 Definición del Umbral de Aprobación

In [ ]:
# Calcular umbral sugerido (percentil 40)
umbral_sugerido = df['Nota_final'].quantile(0.4)
print(f"📊 Umbral sugerido (percentil 40): {umbral_sugerido:.2f}")

# Puedes ajustar este valor
UMBRAL_APROBACION = 14.0  # Ajusta según tu análisis

print(f"\n✅ Umbral seleccionado: {UMBRAL_APROBACION}")

# Verificar distribución
aprobados = (df['Nota_final'] >= UMBRAL_APROBACION).sum()
reprobados = (df['Nota_final'] < UMBRAL_APROBACION).sum()
total = len(df['Nota_final'].dropna())

print(f"\n📊 Distribución con umbral {UMBRAL_APROBACION}:")
print(f"  ✅ Aprobados: {aprobados} ({aprobados/total*100:.1f}%)")
print(f"  ❌ Reprobados: {reprobados} ({reprobados/total*100:.1f}%)")

if aprobados == 0 or reprobados == 0:
    print("\n⚠️ ADVERTENCIA: Solo hay una clase. Ajusta el UMBRAL_APROBACION")

### 3.2 Selección y Preparación de Variables

In [ ]:
# Seleccionar columnas relevantes
# Variables numéricas
variables_numericas = ['Asistencia']  # Ajusta según tus datos

# Variables categóricas
variables_categoricas = ['Nivel', 'Carrera', 'Tipo_Ingreso']  # Ajusta según tus datos

# Crear dataset
columnas_usar = ['Nota_final'] + variables_numericas + variables_categoricas
df_prep = df[columnas_usar].copy()

print(f"📋 Variables seleccionadas: {len(columnas_usar)}")
print(f"  - Numéricas: {variables_numericas}")
print(f"  - Categóricas: {variables_categoricas}")

In [ ]:
# Limpiar valores nulos
nulos_antes = df_prep.isnull().sum().sum()
df_prep = df_prep.dropna()
nulos_despues = df_prep.isnull().sum().sum()

print(f"🧹 Limpieza de datos:")
print(f"  - Nulos eliminados: {nulos_antes}")
print(f"  - Registros finales: {len(df_prep)}")
print(f"  - Nulos restantes: {nulos_despues}")

In [ ]:
# Codificar variables categóricas
le = LabelEncoder()

for col in variables_categoricas:
    if col in df_prep.columns:
        df_prep[col] = le.fit_transform(df_prep[col].astype(str))
        print(f"✅ '{col}' codificada")

print("\n✅ Todas las variables categóricas han sido codificadas")

In [ ]:
# Crear variable objetivo
df_prep['Aprobado'] = (df_prep['Nota_final'] >= UMBRAL_APROBACION).astype(int)

# Verificar balance
distribucion = df_prep['Aprobado'].value_counts()
print("📊 Distribución de clases:")
print(distribucion)
print(f"\n✅ Balance: {min(distribucion)/max(distribucion):.2%}")

# Mostrar muestra
df_prep.head()

## 4. Modelo Supervisado: Clasificación

### 4.1 Preparación de Datos para Entrenamiento

In [ ]:
# Separar características (X) y objetivo (y)
X = df_prep[variables_numericas + variables_categoricas]
y = df_prep['Aprobado']

print(f"📊 Dimensiones:")
print(f"  - X (características): {X.shape}")
print(f"  - y (objetivo): {y.shape}")
print(f"\n📋 Variables predictoras: {list(X.columns)}")

In [ ]:
# División train/test
TEST_SIZE = 0.2
RANDOM_STATE = 42

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE, 
    stratify=y
)

print(f"📊 División de datos (test_size={TEST_SIZE}):")
print(f"  - Entrenamiento: {len(X_train)} muestras")
print(f"  - Prueba: {len(X_test)} muestras")

In [ ]:
# Escalado de características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ Características escaladas con StandardScaler")
print(f"   Media: ~0, Desv. Estándar: ~1")

### 4.2 Entrenamiento del Modelo

In [ ]:
# Modelo 1: Regresión Logística
print("🤖 Entrenando Regresión Logística...")

modelo_lr = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
modelo_lr.fit(X_train_scaled, y_train)

# Predicciones
y_pred_lr = modelo_lr.predict(X_test_scaled)
y_pred_proba_lr = modelo_lr.predict_proba(X_test_scaled)

# Accuracy
accuracy_lr = accuracy_score(y_test, y_pred_lr)

print(f"\n✅ Modelo entrenado")
print(f"🎯 Accuracy: {accuracy_lr:.2%}")

In [ ]:
# Modelo 2: Árbol de Decisión
print("🌳 Entrenando Árbol de Decisión...")

modelo_dt = DecisionTreeClassifier(random_state=RANDOM_STATE)
modelo_dt.fit(X_train_scaled, y_train)

y_pred_dt = modelo_dt.predict(X_test_scaled)
accuracy_dt = accuracy_score(y_test, y_pred_dt)

print(f"✅ Modelo entrenado")
print(f"🎯 Accuracy: {accuracy_dt:.2%}")

In [ ]:
# Modelo 3: Random Forest
print("🌲 Entrenando Random Forest...")

modelo_rf = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=100)
modelo_rf.fit(X_train_scaled, y_train)

y_pred_rf = modelo_rf.predict(X_test_scaled)
accuracy_rf = accuracy_score(y_test, y_pred_rf)

print(f"✅ Modelo entrenado")
print(f"🎯 Accuracy: {accuracy_rf:.2%}")

### 4.3 Evaluación de Modelos

In [ ]:
# Comparación de accuracies
resultados = pd.DataFrame({
    'Modelo': ['Regresión Logística', 'Árbol de Decisión', 'Random Forest'],
    'Accuracy': [accuracy_lr, accuracy_dt, accuracy_rf]
}).sort_values('Accuracy', ascending=False)

print("📊 Comparación de Modelos:")
print(resultados.to_string(index=False))

mejor_modelo = resultados.iloc[0]['Modelo']
mejor_accuracy = resultados.iloc[0]['Accuracy']
print(f"\n🏆 Mejor modelo: {mejor_modelo} ({mejor_accuracy:.2%})")

In [ ]:
# Visualización de comparación
fig, ax = plt.subplots(figsize=(10, 6))

colors = ['#2ecc71' if acc >= 0.9 else '#3498db' if acc >= 0.8 else '#e74c3c' 
          for acc in resultados['Accuracy']]

bars = ax.barh(resultados['Modelo'], resultados['Accuracy'], color=colors, alpha=0.7, edgecolor='black')

# Añadir valores en las barras
for i, (bar, acc) in enumerate(zip(bars, resultados['Accuracy'])):
    ax.text(acc + 0.01, i, f'{acc:.2%}', va='center', fontweight='bold', fontsize=11)

ax.set_xlabel('Accuracy', fontsize=12, fontweight='bold')
ax.set_title('Comparación de Modelos de Clasificación', fontsize=14, fontweight='bold')
ax.set_xlim(0, 1.1)
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

### 4.4 Matriz de Confusión (Mejor Modelo)

In [ ]:
# Usar el mejor modelo (Regresión Logística para este ejemplo)
y_pred_mejor = y_pred_lr

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_mejor)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
           xticklabels=['Reprobado (0)', 'Aprobado (1)'],
           yticklabels=['Reprobado (0)', 'Aprobado (1)'],
           cbar_kws={'label': 'Frecuencia'},
           annot_kws={'size': 16, 'weight': 'bold'})

ax.set_title('Matriz de Confusión - Regresión Logística', fontsize=14, fontweight='bold')
ax.set_ylabel('Valor Real', fontsize=12)
ax.set_xlabel('Predicción', fontsize=12)

plt.tight_layout()
plt.show()

print("\n📊 Interpretación:")
print(f"  - Verdaderos Negativos (TN): {cm[0,0]} - Reprobados correctamente predichos")
print(f"  - Falsos Positivos (FP): {cm[0,1]} - Reprobados predichos como aprobados")
print(f"  - Falsos Negativos (FN): {cm[1,0]} - Aprobados predichos como reprobados")
print(f"  - Verdaderos Positivos (TP): {cm[1,1]} - Aprobados correctamente predichos")

### 4.5 Reporte de Clasificación

In [ ]:
# Reporte detallado
print("📋 Reporte de Clasificación:")
print("="*60)
print(classification_report(y_test, y_pred_mejor, 
                          target_names=['Reprobado', 'Aprobado']))

# Como DataFrame
report_dict = classification_report(y_test, y_pred_mejor, 
                                   target_names=['Reprobado', 'Aprobado'],
                                   output_dict=True)
report_df = pd.DataFrame(report_dict).transpose()
report_df

### 4.6 Importancia de Variables

In [ ]:
# Importancia de características (Regresión Logística)
importancias = pd.DataFrame({
    'Variable': X.columns,
    'Coeficiente': modelo_lr.coef_[0],
    'Importancia_Abs': np.abs(modelo_lr.coef_[0])
}).sort_values('Importancia_Abs', ascending=False)

print("📊 Importancia de Variables:")
print(importancias[['Variable', 'Coeficiente']])

# Visualización
fig, ax = plt.subplots(figsize=(10, 6))
colors = ['green' if x > 0 else 'red' for x in importancias['Coeficiente']]
ax.barh(importancias['Variable'], importancias['Coeficiente'], color=colors, alpha=0.7, edgecolor='black')
ax.set_xlabel('Coeficiente', fontsize=12)
ax.set_title('Importancia de Variables\n(Verde: Influencia Positiva | Rojo: Influencia Negativa)', 
            fontsize=14, fontweight='bold')
ax.axvline(x=0, color='black', linestyle='--', linewidth=1)
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Modelo No Supervisado: Clustering (K-means)

### 5.1 Preparación de Datos para Clustering

In [ ]:
# Seleccionar variables para clustering
VAR_X = 'Asistencia'
VAR_Y = 'Nota_final'

df_cluster = df[[VAR_X, VAR_Y]].dropna()

print(f"📊 Variables para clustering:")
print(f"  - X: {VAR_X}")
print(f"  - Y: {VAR_Y}")
print(f"  - Registros: {len(df_cluster)}")

In [ ]:
# Escalado
scaler_cluster = StandardScaler()
X_cluster_scaled = scaler_cluster.fit_transform(df_cluster)

print("✅ Datos escalados para clustering")

### 5.2 Aplicación de K-means

In [ ]:
# K-means con 3 clusters
N_CLUSTERS = 3

print(f"🔍 Aplicando K-means con {N_CLUSTERS} clusters...")

kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_cluster_scaled)

df_cluster['Cluster'] = clusters

print(f"✅ Clustering completado")
print(f"\n📊 Distribución de clusters:")
print(df_cluster['Cluster'].value_counts().sort_index())

### 5.3 Visualización de Clusters

In [ ]:
# Visualización principal
fig, ax = plt.subplots(figsize=(12, 8))

scatter = ax.scatter(df_cluster[VAR_X], df_cluster[VAR_Y], 
                     c=clusters, cmap='viridis', alpha=0.6, s=100, 
                     edgecolors='black', linewidth=0.5)

# Centroides
centroides = scaler_cluster.inverse_transform(kmeans.cluster_centers_)
ax.scatter(centroides[:, 0], centroides[:, 1], 
          c='red', marker='X', s=500, edgecolors='black',
          linewidths=3, label='Centroides', zorder=5)

# Etiquetar centroides
for i, (x, y) in enumerate(centroides):
    ax.annotate(f'C{i}', (x, y), fontsize=14, fontweight='bold', 
               color='white', ha='center', va='center')

ax.set_xlabel(VAR_X, fontsize=12, fontweight='bold')
ax.set_ylabel(VAR_Y, fontsize=12, fontweight='bold')
ax.set_title(f'Clustering K-means (k={N_CLUSTERS})', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Cluster', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

### 5.4 Análisis de Cada Cluster

In [ ]:
# Estadísticas por cluster
print("📊 Análisis Detallado por Cluster:")
print("="*60)

for i in range(N_CLUSTERS):
    cluster_data = df_cluster[df_cluster['Cluster'] == i]
    n_estudiantes = len(cluster_data)
    
    print(f"\n🔹 Cluster {i} - {n_estudiantes} estudiantes")
    print(f"  {VAR_X}:")
    print(f"    - Promedio: {cluster_data[VAR_X].mean():.2f}")
    print(f"    - Desv. Est: {cluster_data[VAR_X].std():.2f}")
    print(f"    - Min: {cluster_data[VAR_X].min():.2f}")
    print(f"    - Max: {cluster_data[VAR_X].max():.2f}")
    
    print(f"  {VAR_Y}:")
    print(f"    - Promedio: {cluster_data[VAR_Y].mean():.2f}")
    print(f"    - Desv. Est: {cluster_data[VAR_Y].std():.2f}")
    print(f"    - Min: {cluster_data[VAR_Y].min():.2f}")
    print(f"    - Max: {cluster_data[VAR_Y].max():.2f}")
    
    # Interpretación
    if cluster_data[VAR_Y].mean() > df_cluster[VAR_Y].mean():
        if cluster_data[VAR_X].mean() > df_cluster[VAR_X].mean():
            perfil = "✅ Alto rendimiento con buena asistencia"
        else:
            perfil = "📊 Buen rendimiento pero asistencia mejorable"
    else:
        if cluster_data[VAR_X].mean() < df_cluster[VAR_X].mean():
            perfil = "⚠️ Bajo rendimiento y baja asistencia (REQUIERE ATENCIÓN)"
        else:
            perfil = "📉 Asistencia aceptable pero bajo rendimiento"
    
    print(f"  📋 Perfil: {perfil}")

In [ ]:
# Resumen estadístico como DataFrame
resumen_clusters = df_cluster.groupby('Cluster')[[VAR_X, VAR_Y]].agg(['mean', 'std', 'min', 'max'])
print("\n📊 Resumen Estadístico por Cluster:")
resumen_clusters

### 5.5 Método del Codo (Elbow)

In [ ]:
# Evaluar diferentes valores de k
inertias = []
K_range = range(2, 8)

for k in K_range:
    km_temp = KMeans(n_clusters=k, random_state=42, n_init=10)
    km_temp.fit(X_cluster_scaled)
    inertias.append(km_temp.inertia_)

# Visualización
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
ax.axvline(x=N_CLUSTERS, color='red', linestyle='--', linewidth=2, 
          label=f'k seleccionado = {N_CLUSTERS}')
ax.set_xlabel('Número de Clusters (k)', fontsize=12, fontweight='bold')
ax.set_ylabel('Inercia', fontsize=12, fontweight='bold')
ax.set_title('Método del Codo para K-means', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

## 6. Comparación de Modelos

### 6.1 Resumen de Resultados

In [ ]:
print("="*70)
print("📊 RESUMEN DE MODELOS IMPLEMENTADOS")
print("="*70)

print("\n🤖 MODELO SUPERVISADO: CLASIFICACIÓN")
print("-" * 70)
print(f"  Algoritmo: Regresión Logística")
print(f"  Accuracy: {accuracy_lr:.2%}")
print(f"  Objetivo: Predecir Aprobado/Reprobado")
print(f"  Variables: {', '.join(X.columns)}")

print("\n🔍 MODELO NO SUPERVISADO: CLUSTERING")
print("-" * 70)
print(f"  Algoritmo: K-means")
print(f"  Número de clusters: {N_CLUSTERS}")
print(f"  Objetivo: Agrupar estudiantes por patrones")
print(f"  Variables: {VAR_X}, {VAR_Y}")
print(f"  Distribución: {df_cluster['Cluster'].value_counts().to_dict()}")

print("\n" + "="*70)

### 6.2 ¿Qué Modelo es Mejor?

In [ ]:
print("""\n💡 COMPARACIÓN Y CONCLUSIONES
="*70

❓ ¿Qué modelo es mejor?

No hay un modelo "mejor" en términos absolutos. Cada uno sirve para 
propósitos DIFERENTES:

🤖 MODELO SUPERVISADO (Clasificación):
  ✅ Ventajas:
    - Predice resultados específicos (Aprobado/Reprobado)
    - Permite medir accuracy y otras métricas cuantitativas
    - Útil para predicciones futuras de nuevos estudiantes
    - Identifica qué variables influyen más en el resultado
  
  ❌ Limitaciones:
    - Requiere datos etiquetados (conocer el resultado)
    - Depende de la calidad del umbral definido
    - Puede no capturar patrones complejos no lineales

🔍 MODELO NO SUPERVISADO (Clustering):
  ✅ Ventajas:
    - Descubre patrones ocultos sin etiquetas previas
    - Identifica grupos naturales de estudiantes
    - Útil para segmentación y perfiles
    - No requiere conocer el resultado de antemano
  
  ❌ Limitaciones:
    - No predice valores específicos
    - Interpretación más subjetiva
    - No tiene métricas de precisión directas
    - Requiere definir número de clusters manualmente

🔄 RECOMENDACIÓN: USO COMPLEMENTARIO

La mejor estrategia es usar AMBOS modelos:

1. Clustering → Identifica grupos de riesgo o perfiles
2. Clasificación → Predice resultados individuales
3. Intervención → Acciones personalizadas según cluster + predicción

📊 EJEMPLO PRÁCTICO:
  - Clustering detecta: "Grupo de estudiantes con baja asistencia"
  - Clasificación predice: "Juan tiene 85% probabilidad de reprobar"
  - Acción: Tutorías focalizadas para Juan y su grupo

="*70
""")

## 7. Conclusiones

In [ ]:
print("""\n📝 CONCLUSIONES DE LA PRÁCTICA
="*70

✅ LOGROS OBTENIDOS:

1. Dataset preparado exitosamente:
   - {} registros limpios
   - {} variables predictoras
   - Balance de clases: {:.1%}

2. Modelo Supervisado implementado:
   - Accuracy alcanzado: {:.2%}
   - Desempeño: {}
   - Variables más influyentes identificadas

3. Clustering aplicado:
   - {} grupos de estudiantes identificados
   - Perfiles claramente diferenciados
   - Grupos de riesgo detectados

📊 HALLAZGOS PRINCIPALES:

- La {} es la variable más importante para predecir aprobación
- Se identificaron {} perfiles distintos de estudiantes
- Los estudiantes con {} requieren atención especial

💡 APLICACIONES PRÁCTICAS:

1. Sistema de alerta temprana de reprobación
2. Segmentación para tutorías personalizadas
3. Identificación de factores de riesgo académico
4. Optimización de recursos educativos

🎓 APRENDIZAJES:

- Implementación de modelos supervisados y no supervisados
- Evaluación de modelos con métricas apropiadas
- Interpretación de resultados para toma de decisiones
- Uso complementario de diferentes técnicas de ML

="*70
""".format(
    len(df_prep),
    len(X.columns),
    min(distribucion)/max(distribucion),
    accuracy_lr,
    'Excelente' if accuracy_lr >= 0.9 else 'Bueno' if accuracy_lr >= 0.8 else 'Aceptable',
    N_CLUSTERS,
    importancias.iloc[0]['Variable'],
    N_CLUSTERS,
    "baja asistencia y bajo rendimiento"
))

## 8. Exportar Resultados

In [ ]:
# Guardar resultados en CSV
# Predicciones del modelo supervisado
resultados_pred = pd.DataFrame({
    'y_real': y_test,
    'y_pred': y_pred_mejor,
    'prob_reprobado': y_pred_proba_lr[:, 0],
    'prob_aprobado': y_pred_proba_lr[:, 1]
})
resultados_pred.to_csv('resultados_predicciones.csv', index=False)
print("✅ Predicciones guardadas en 'resultados_predicciones.csv'")

# Resultados del clustering
df_cluster.to_csv('resultados_clustering.csv', index=False)
print("✅ Clustering guardado en 'resultados_clustering.csv'")

print("\n🎉 ¡Práctica completada exitosamente!")